# Example: Feature-centric analogous bars
* This notebook shows an application of the <b>feature-centric analogous bars method</b> to compare barcodes built on two different point clouds `P` and `Q`.

#### Feature-centric analogous bars
* <b> Inputs </b>
    * Distance matrix among points in `P`
    * Distance matrix among points in `Q`
    * Corss-system distance matrix among `P` and `Q`.
* <b> Goal </b>
    * Given a selected bar in the Vietoris-Rips barcode `barcode(VR(P))`, find its representations in `barcode(VR(Q))`
* <b> Implementation </b>:
    * User selects a bar of interest from `barcode(VR(P))`
    * The function `run_extension_VR_to_W_bar` finds the representation of the selected bar in the Witness complex `W(P,Q)`.
    * We then apply Dowker's theorem to find the corresponding cycle in `W(Q,P)`.
    * Lastly, we run the function `run_extension_W_to_VR_bar` to find the bar extension in `barcode(VR(Q))`.
    * Note that we don't provide an individual function for feature-centric analogous bars method. 
    * All extension methds are implemented component-wise with $\mathbb{F}_2$ coefficients. We assume that all bars of barcodes have unique death times. 
* <b> Comparison to Algorithm 6 of paper </b>
    * In the paper, we run the first extension method (from VR to W) and find the collection of all cycle extensions $E(\tau, W^\bullet_{P,Q})$. We then apply Dowker's Theorem to all such cycle extensions to find the collection $E(\tau, W^\bullet_{Q,P})$. For each cycle $[\sigma] \in E(\tau, W^\bullet_{Q,P})$, we run the second extension method (from W to VR).   
    * In our implementation, we don't apply the second extension method to the entire collection $E(\tau, W^\bullet_{Q,P})$ as this can require huge computation time. Instead, we allow the user to select a specific cycle in $E(\tau, W^\bullet_{P,Q})$ via `cycle_W_PQ`. We then find its corresponding cycle `cycle_W_QP` in $W^\bullet_{Q,P}$ via Dowker's Theorem and run the second extension method on `cycle_W_QP`. 
   

#### Example data 
* `P` and `Q` are both points sampled from a torus. 
* The goal is to identify the cycles in `P` and `Q`

#### Contents 
1. Load points and visualize
2. Plot the four relevant barcodes
3. Run the extension method from VR to W
4. Apply Dowker's Theorem
5. Run the extension method from W to VR. 
6. Explore cycle extension & bar extension under fixed interval decompositions of `barcode(VR(Q))`.
7. Explore alternative bar extensions under all possible interval decompositions of `barcode(VR(Q))`.

In [3]:
using Revise
includet("../../../extension_method.jl")

In [5]:
using .ext
using DelimitedFiles
using HDF5
using Printf
using Eirene
using Plots
using JLD

# 1. Load points and visualize
`P`, `Q`: points sampled from the torus

In [6]:
# load
data = load("data.jld")
P = data["P"] # 3 dimensional coordinates
P_theta = data["P_theta"]
P_phi = data["P_phi"]
Q = data["Q"] # 3 dimensional coordinates
Q_theta = data["Q_theta"]
Q_phi = data["Q_phi"];

print("number of points in P: ", size(P,1), "\n")
print("number of points in Q: ", size(Q,1))

# Assign colors to visualization
P_color = "#ff8d00"
Q_color = "#008181";

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


number of points in P: 150
number of points in Q: 150

In [ ]:
# The points were generated using the following code
"""
# parameters of a torus
R = 2
r = 1
n = 150

# P 
P, P_theta, P_phi = ext.sample_torus(R, r, n, return_angles = true)

# Q
Q, Q_theta, Q_phi = ext.sample_torus(R, r, n, return_angles = true)

# save points
# save("data.jld", "P", P, "P_theta", P_theta, "P_phi", P_phi,
#    "Q", Q, "Q_theta", Q_theta, "Q_phi", Q_phi);
"""

In [10]:
p = plot_3D(P,Q)
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl

Plot the points on square torus. Assume that the sides of the squares have been identified to represent a torus

In [12]:
P_2d = hcat(P_theta, P_phi)
Q_2d = hcat(Q_theta, Q_phi)
p = plot_P_Q(P_2d, Q_2d)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Plot the four relevant barcodes
For this example, we'll use the distances computed on the square torus

In [13]:
# compute distance matrices
all_theta = vcat(P_theta, Q_theta)
all_phi = vcat(P_phi, Q_phi);
D = compute_distance_square_torus(all_theta, all_phi)
n = size(P_theta)[1]

# get submatrices
D_P = D[1:n, 1:n]
D_Q = D[n+1:end, n+1:end]
D_P_Q = D[1:n, n+1:end]
D_Q_P = D[n+1:end, 1:n];

In [15]:
# Compute Vietoris-Rips persistence on two regions
dim = 1
VR_P = eirene(D_P, record = "all", maxdim = dim)
VR_Q = eirene(D_Q, record = "all", maxdim = dim)

# compute Witness persistence
W_P = compute_Witness_persistence(D_P_Q, maxdim = dim)
W_Q = compute_Witness_persistence(D_Q_P, maxdim = dim);

In [17]:
# plot all four barcodes
barcode_VR_P = barcode(VR_P, dim = 1)
barcode_W_P = barcode(W_P["eirene_output"], dim = 1)
barcode_W_Q = barcode(W_Q["eirene_output"], dim = 1)
barcode_VR_Q = barcode(VR_Q, dim = 1)

# plot
p1 = plot_barcode(barcode_VR_P, lw = 2, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_W_P, lw = 2, title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_W_Q, lw = 2, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode_VR_Q, lw = 2, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = grid(4,1), size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

# 3. Run the extension method from VR to W

Given the selected interval in `barcode(VR(P))`, find its cycle extensions in `W(P,Q)`

In [18]:
# select bar of interest
VR_P_class = 45

45

In [24]:
# plot cycle rep of selected bar
cycle = get_multiclass_cyclerep(VR_P, VR_P_class)
plot_cycle_square_torus(P_2d, Q_2d, cycle = cycle, size = (300, 300))

<!DOCTYPE html>
 
 
 Plots.jl

In [25]:
# run extension method
extension_to_W_PQ = run_extension_VR_to_W_bar(C_VR = VR_P,
                                              D_VR = D_P,
                                              VR_bar = VR_P_class,
                                              W = W_P,
                                              D_W = D_P_Q);

### Find cycle extensions at specific parameter

In [29]:
param = extension_to_W_PQ["epsilon_0"]
CE_param, _ = find_CE_BE_at_param(extension_to_W_PQ, param);

In [30]:
# select a cycle extension
cycle_W_PQ = CE_param[0];

In [32]:
# plot selected cycle extension
plot_cycle_square_torus(P_2d, Q_2d, cycle = cycle_W_PQ, size = (300, 300))

<!DOCTYPE html>
 
 
 Plots.jl

# 4. Find corresponding cycle in `W(Q,P)` via Dowker's Theorem

In [33]:
# find corresponding cycle
cycle_W_QP = find_Dowker_cycle_correspondence(cycle_W_PQ, param, D_P_Q);

In [35]:
# plot
plot_cycle_square_torus(P_2d, Q_2d, cycle = cycle_W_QP, cycle_loc = "Q")

<!DOCTYPE html>
 
 
 Plots.jl

# 5. Run the extension method from `W(Q,P)` to `VR(Q)`

Find parameter at which to run the extension method.
We'll choose the parameter immediately prior to the death time of `cycle_W_QP`.

In [36]:
# choose parameter
parameter = find_cycle_death_in_Witness(cycle_W_QP, W_Q)
psi = maximum(D_Q_P[D_Q_P.< parameter])

1.629996315770294

In [37]:
# run extension
extension_to_VR_Q = run_extension_W_to_VR(W = W_Q,
                                          W_cycle = cycle_W_QP,
                                          psi = psi,
                                          C_VR = VR_Q,
                                          D_VR = D_Q);

### Explore results of the second extension method.

In [38]:
plot_pY(extension_to_VR_Q)

<!DOCTYPE html>
 
 
 Plots.jl

In [39]:
p = return_extension_results_at_parameter(extension_to_VR_Q)

*** Parameter key, value pair *** 
key: 1 parameter: 0.576112 
key: 2 parameter: 0.577768 
key: 3 parameter: 0.579211 
key: 4 parameter: 0.582741 
key: 5 parameter: 0.603680 
key: 6 parameter: 0.606564 
key: 7 parameter: 0.628708 
key: 8 parameter: 0.631464 
key: 9 parameter: 0.643448 
key: 10 parameter: 0.646555 
key: 11 parameter: 0.672947 
key: 12 parameter: 0.678108 
key: 13 parameter: 0.711588 
key: 14 parameter: 0.739252 
key: 15 parameter: 0.756156 
key: 16 parameter: 0.760930 
key: 17 parameter: 0.779900 
key: 18 parameter: 0.781234 
key: 19 parameter: 0.820879 
key: 20 parameter: 0.851726 
key: 21 parameter: 0.860603 
key: 22 parameter: 0.879290 
key: 23 parameter: 0.884756 
key: 24 parameter: 0.894657 
key: 25 parameter: 0.914532 
key: 26 parameter: 0.946238 
key: 27 parameter: 0.956688 
key: 28 parameter: 0.970763 
key: 29 parameter: 1.098567 



Select a key for parameter 1


Selected parameter: 0.5761124664956675

Baseline bars extension at selected parameter: [40]

*** Offset bar extensions at selected parameter *** 
key: 1 offset bar extension: [15]
key: 2 offset bar extension: [16]
key: 3 offset bar extension: [12]
key: 4 offset bar extension: [39]
key: 5 offset bar extension: [13]



Select keys for offset bar extensions. 
Leave blank to select none. 
To select multiple keys, separate keys with space. ex) 1 2 3 :  



Baseline bars extension at selected parameter: [40]


<!DOCTYPE html>
 
 
 Plots.jl

# 6. Explore the bar extension under fixed interval decompositions of `barcode(VR(Q))`.

* We use the function `find_CE_BE_at_param()` to find all cycle extensions and bar extensions at a specific parameter.

In [40]:
# select parameter
param = extension_to_VR_Q["nontrivial_pY"][1]
CE_param, BE_param = find_CE_BE_at_param(extension_to_VR_Q, param);

<b> plot cycle extensions to `barcode(VR(Q))`</b>

In [41]:
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE_param))

number of cycle extensions at parameter 0.5761 : 32

In [43]:
# plot a cycle extension at selected parameter
idx = 0
p = plot_cycle_square_torus(P_2d, Q_2d, cycle = CE_param[idx], cycle_loc = "Q", title = "cycle extension",
                                P_markersize = 5, Q_markersize = 5; legend = false)

<!DOCTYPE html>
 
 
 Plots.jl

The cycle above looks discontinuous. However, this is due to the fact that our function doesn't plot any 1-simplices that crosses the boundary of the square torus.

Plot the <b>bar extensions</b> at given parameter.
* Select a cycle extension 
* Find and plot the corresponding bar extensions

In [44]:
# select parameter 
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE_param))

number of cycle extensions at parameter 0.5761 : 32

In [45]:
# select cycle extension 
y= 0

# find the corresponding bar extension
be = BE_param[y]

# plot the bar extension
barcode_VR_Q = barcode(VR_Q, dim = dim)
p = plot_barcode(barcode_VR_Q, title = "selected bar extension to barcode(VR(Q))", lw = 2, selected_bars = be, epsilon= param, v_line = [param])
plot(p, size = (500, 300))

<!DOCTYPE html>
 
 
 Plots.jl

Plot a summary analogous bars plot under fixed interval decomposition of `barcode(VR(Q))`

In [47]:
# plot the summary
l = grid(4,1)
p1 = plot_barcode(barcode_VR_P, selected_bars = [VR_P_class], title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_W_P , title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_W_Q, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode_VR_Q, title = "Barcode(VR(Q))", titlefontsize = 12, selected_bars = be)
plot(p1, p2, p3, p4, layout = l, size = (500, 800))


<!DOCTYPE html>
 
 
 Plots.jl

# 7. Explore alternative bar extensions under all possible interval decompositions of `barcode(VR(Q))`.

* Up to this point, the bar extension result has been obtained for some fixed interval decomposition of `barcode(VR(Q))`.
* In this section, we show how to find the bar extensions under all possible interval decompositions. Given `cycle_W_QP` (corresponds to $[\sigma] \in E(\tau, W^\bullet_{Q,P})$ in the paper), the goal is to find $S([\sigma], X^\bullet_Q)$ from Algorithm 6 of paper. We'll refer to this set as <b>alternative bar extensions</b> since these arise from alternative choices of the interval decompositions.
* There are three different methods for exploring the alternative bar extensions. The appropriate tool depends on the sizes of the barcodes of the auxiliary filtration and the target filtration. 

1. Find all alternative bar extensions for all parameters.  
    * This is recommended for data with small barcodes. 
    * This finds the full $S([\sigma], Y^{\bullet}) = \{ S^{\mathcal{D} \circ L^{-1}}_{[y]} | \ell \in p_Y, [y] \in \mathfrak{E}_{\ell}, L \in L_Y \}$ in Algorithm 3 of paper.
2. Find alternative bar extensions at specific parameters.  
    * This is recommended for data with medium size barcodes.
    * Given a parameter $\ell$, this method finds $S([\sigma], Y^{\bullet}; \ell) = \{ S^{\mathcal{D} \circ L^{-1}}_{[y]} | [y] \in \mathfrak{E}_{\ell}, L \in L_Y \} $
3. Find alternative bar extensions of a specific bar extension.
    * This is recommended for data with large size barcodes.
    * Given a selected parameter $\ell$ and cycle extension $[y] \in \mathfrak{E}_{\ell}$, this method finds $\{S^{\mathcal{D} \circ L^{-1}}_{[y]} | L \in L_Y \}$. 
    
For this example, we'll implement method 3 due to the large number of bars `barcode(VR(Q))`. Depending on the selected parameter, the function `find_alternative_bar_extension()` can lead to an out of memory error. This will happen when a user selects a parameter at which there are numerous bars in the barcode. 

For example implementations of methods 1 and 2, see notebook `EXAMPLE_EXTENSION_VR_VR.ipynb`


In [48]:
# select a parameter and bar extension
p = return_extension_results_at_parameter(extension_to_VR_Q)

*** Parameter key, value pair *** 
key: 1 parameter: 0.576112 
key: 2 parameter: 0.577768 
key: 3 parameter: 0.579211 
key: 4 parameter: 0.582741 
key: 5 parameter: 0.603680 
key: 6 parameter: 0.606564 
key: 7 parameter: 0.628708 
key: 8 parameter: 0.631464 
key: 9 parameter: 0.643448 
key: 10 parameter: 0.646555 
key: 11 parameter: 0.672947 
key: 12 parameter: 0.678108 
key: 13 parameter: 0.711588 
key: 14 parameter: 0.739252 
key: 15 parameter: 0.756156 
key: 16 parameter: 0.760930 
key: 17 parameter: 0.779900 
key: 18 parameter: 0.781234 
key: 19 parameter: 0.820879 
key: 20 parameter: 0.851726 
key: 21 parameter: 0.860603 
key: 22 parameter: 0.879290 
key: 23 parameter: 0.884756 
key: 24 parameter: 0.894657 
key: 25 parameter: 0.914532 
key: 26 parameter: 0.946238 
key: 27 parameter: 0.956688 
key: 28 parameter: 0.970763 
key: 29 parameter: 1.098567 



Select a key for parameter 1


Selected parameter: 0.5761124664956675

Baseline bars extension at selected parameter: [40]

*** Offset bar extensions at selected parameter *** 
key: 1 offset bar extension: [15]
key: 2 offset bar extension: [16]
key: 3 offset bar extension: [12]
key: 4 offset bar extension: [39]
key: 5 offset bar extension: [13]



Select keys for offset bar extensions. 
Leave blank to select none. 
To select multiple keys, separate keys with space. ex) 1 2 3 :  



Baseline bars extension at selected parameter: [40]


<!DOCTYPE html>
 
 
 Plots.jl

In [49]:
param = param = extension_to_VR_Q["nontrivial_pY"][1]
bar_ext = [40]

1-element Array{Int64,1}:
 40

In [50]:
# find alternative representations of the selected bar extension
alt_bar_ext = find_alternative_bar_extension(extension_to_VR_Q, param, bar_extension = bar_ext)

32-element Array{Array{Int64,1},1}:
 [40]
 [12, 40]
 [13, 40]
 [15, 40]
 [16, 40]
 [39, 40]
 [12, 13, 40]
 [12, 15, 40]
 [12, 16, 40]
 [12, 39, 40]
 [13, 15, 40]
 [13, 16, 40]
 [13, 39, 40]
 ⋮
 [12, 15, 39, 40]
 [12, 16, 39, 40]
 [13, 15, 16, 40]
 [13, 15, 39, 40]
 [13, 16, 39, 40]
 [15, 16, 39, 40]
 [12, 13, 15, 16, 40]
 [12, 13, 15, 39, 40]
 [12, 13, 16, 39, 40]
 [12, 15, 16, 39, 40]
 [13, 15, 16, 39, 40]
 [12, 13, 15, 16, 39, 40]

In [53]:
# plot one of the alternative bar extensions

# select an alternative bar extension
alt = alt_bar_ext[10]

barcode_Y = barcode(extension_to_VR_Q["C_VR"], dim = dim)
p =plot_barcode(barcode_Y, selected_bars = alt,
                    epsilon = param, v_line = [param],
                    title = "alternative bar extension")
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl